<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250, style="padding: 10px"> 
<b>Intermediate TAP Queries for DP0 catalogs </b> <br>
Last verified to run on 2021-06-25 with LSST Science Pipelines release w_2021_25 <br>
Contact authors: Leanne Guy <br>
Target audience: All DP0 delegates. <br>
Container Size: medium <br>
Questions welcome at <a href="https://community.lsst.org/c/support/dp0">community.lsst.org/c/support/dp0</a> <br>
Find DP0 documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a> <br>

**Credit:** Originally developed by Leanne Guy in the context of the Rubin DP0.1. Please consider acknowledging Leanne Guy if this notebook is used for the preparation of journal articles or software releases.

### Learning Objectives

The Rubin Science Platform provides QUERY access to the DP0.1 catalogs via TAP from jupyter notebooks. TAP is a Virtual Observatory protocol for access to catalog data. In this tutorial, we will learn how to explore the DP0.1 archive via TAP and execute complex queries to retrieve data. Full TAP documentation can be found [here](https://www.ivoa.net/documents/TAP/).

This notebook demonstrates how to:<br>
1. Explore the DP0.1 schema and catalogs using the Rubin TAP service<br>
2. Query the DP0.1 Object and Truth Match catalogs, and retrieve data for analysis<br>
3. Visualize and interact with the retrieved data set<br>
4. Work with asynchronous TAP queries

### Set Up 

In [1]:
# Import general python packages
import numpy as np
import re
import pandas
from pandas.testing import assert_frame_equal

# Astropy
from astropy import units as u
from astropy.coordinates import SkyCoord

# Bokeh for interactive visualization
import bokeh
from bokeh.io import output_file, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.plotting import figure
from bokeh.transform import factor_cmap

import holoviews as hv

# Set the maximum number of rows to display from pandas
pandas.set_option('display.max_rows', 20)

# Configure bokeh to generate output in notebook cells when show() is called. 
output_notebook()

Loading BokehJS ...

In [2]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### 1. Explore the DP0.1 schema 

#### 1.1 Create the Rubin TAP Service client

Table Access Procotol (TAP) provides standardized access to catalog data for discovery, search, and retrieval. Full <a href="http://www.ivoa.net/documents/TAP">documentation for TAP</a> is provided by the International Virtual Observatory Alliance (IVOA).

The TAP service uses a query language similar to SQL (Structured Query Langage) called ADQL (Astronomical Data Query Language). The <a href="http://www.ivoa.net/documents/latest/ADQL.html">documentation for ADQL</a> includes more information about syntax and keywords.

**Hazard Warning:** Not all ADQL functionality is supported yet in the DP0 RSP.

In [3]:
# Import the Rubin TAP service utilities 
from rubin_jupyter_utils.lab.notebook import get_tap_service, retrieve_query    

# Get an instance of the TAP service 
service = get_tap_service()

assert service is not None
assert service.baseurl == "https://data.lsst.cloud/api/tap"

Patching auth into notebook.base.handlers.IPythonHandler(notebook.base.handlers.AuthenticatedHandler) -> IPythonHandler(jupyterhub.singleuser.mixins.HubAuthenticatedHandler, notebook.base.handlers.AuthenticatedHandler)


#### 1.2 Schema discovery

To find out what schemas, tables and columns exist, we will query the Rubin TAP schema. 

In [4]:
# Query to find out what schemas are in the Rubin TAP_SCHEMA
query = "SELECT * FROM tap_schema.schemas"

# Execute the query. 
results = service.search(query)

# A TAP Results object is returned
print(type(results))

<class 'pyvo.dal.tap.TAPResults'>


In [5]:
# Convert the results to an astropy table and display
results = service.search(query).to_table()
results

description,schema_index,schema_name,utype
str512,int32,str64,str512
Data Preview 0.1 includes five tables based on the DESC's Data Challenge 2 simulation of 300 square degrees of the wide-fast-deep LSST survey region after 5 years. All tables contain objects detected in coadded images.,0,dp01_dc2_catalogs,
A TAP-standard-mandated schema to describe tablesets in a TAP 1.1 service,100000,tap_schema,
UWS Metadata,120000,uws,


#### 1.3 DC2 tables (catalogs) in DP0.1

All the DP0 tables (catalogs) are in the "dp01_dc2_catalogs" schema (table collection). We can programatically extract the DP0.1 schema_name into a variable.

In [6]:
# Find the DP0 schema name and store as a variable
schema_names = results['schema_name']
for name in schema_names:
    if re.search('dp01', name):
        dp01_schema_name = name
        break
print("DP0.1 schema is " + dp01_schema_name)

DP0.1 schema is dp01_dc2_catalogs


Let's explore tables in the DP0.1 schema, ordering them by their database.  This is the order in which they will appear presented to the user in the RSP Portal. We see the five tables in the DP0.1 schema, the same five tables that are presented via the Portal GUI, together with a description of each. 

In [7]:
# Prepare the query to explore the tables in the DP0.1 schema
query = "SELECT * FROM tap_schema.tables "\
        "WHERE tap_schema.tables.schema_name = '" \
        + dp01_schema_name + "' order by table_index ASC"
print(query)

SELECT * FROM tap_schema.tables WHERE tap_schema.tables.schema_name = 'dp01_dc2_catalogs' order by table_index ASC


In [8]:
# Execute the query
results = service.search(query)
results = results.to_table()
results

description,schema_name,table_index,table_name,table_type,utype
str512,str512,int32,str64,str8,str512
The object table from the DESC DC2 simulated sky survey as described in arXiv:2101.04855. Includes astrometric and photometric parameters for objects detected in coadded images. (137 columns),dp01_dc2_catalogs,0,dp01_dc2_catalogs.object,table,
The truth-match table for the DESC DC2's object table as described in arXiv:2101.04855. Includes the noiseless astrometric and photometric parameters and the best matches to the object table. (30 columns),dp01_dc2_catalogs,1,dp01_dc2_catalogs.truth_match,table,
"Select astrometry-related parameters for objects detected in the coadded images, such as coordinates, footprints, patch/tract information, and deblending parameters. (29 columns)",dp01_dc2_catalogs,2,dp01_dc2_catalogs.position,table,
"Measurements for objects detected in the coadded images, including photometry, astrometry, shape, deblending, model fits, and related background and flag parameters. This table joined with the position table is very similar to the object table, but with additional columns. (236 columns)",dp01_dc2_catalogs,3,dp01_dc2_catalogs.reference,table,
"Forced photometry measurements for objects detected in the coadded images, at the locations defined by the position table. (747 columns)",dp01_dc2_catalogs,4,dp01_dc2_catalogs.forced_photometry,table,


<br>
Here are some definitions to help delegates understand the contents of the TAP schema. 

* `schema` - database terminology for the abstract design that represents the storage of data in a database. 
* `tap_schema` - the specific schema describing the TAP service. All TAP services must support a set of tables in a schema named TAP_SCHEMA that describe the tables and columns included in the service.
* `table` - a collection of related data held in a table format in a database, e.g., the object(dp01_dc2_catalogs.object) or position (dp01_dc2_catalogs.position) tables 
* `table collection` - a collection of tables. e.g., `dp01_dc2_catalogs`	
* `results` - the query result set. The TAP service returns data from a query as a `TAPResults` object. Find more about `TAPResults` [here](https://pyvo.readthedocs.io/en/latest/api/pyvo.dal.TAPResults.html).

### 2. Querying the DP0.1 Object and Truth Match catalogs

The Object catalog (dp01_dc2_catalogs.object) contains sources detected in the coadded images (also called stacked or combined images). The Object catalog is likely to be the catalog that is of the most interest to DP0 delgates. 

The `object` catalog is described in the <a href="https://arxiv.org/abs/2101.04855">DESC's DC2 data release note</a>, and more information about the simulated data can be found in the <a href="https://ui.adsabs.harvard.edu/abs/2021ApJS..253...31L/abstract">DESC's DC2 paper</a>. 

#### 2.1 Specifying the maximum number of records to return
For debugging and testing queries, it is often useful to only return a few records for expediency. This can be done in one of two ways, setting the `TOP` field in a query, or setting the `maxrec` parameter in the TAP service query. The two methods are identical. 

In [9]:
# Define the maximum records to return
max_rec = 5

# Build a query to find object with extendedness = 0 and sort the returned
# result set by decreasing magnitude in the r band.
# Only return the first 5 results
query = "SELECT TOP " + str(max_rec) + \
        " objectId, ra, dec, extendedness, mag_r, magerr_r, good " \
        "FROM dp01_dc2_catalogs.object " \
        "WHERE extendedness = 0 " \
        "AND mag_r < 24 " \
        "ORDER by mag_r DESC"
print(query)

SELECT TOP 5 objectId, ra, dec, extendedness, mag_r, magerr_r, good FROM dp01_dc2_catalogs.object WHERE extendedness = 0 AND mag_r < 24 ORDER by mag_r DESC


In [10]:
# Execute the query
results = service.search(query)
assert len(results) == max_rec

In [11]:
# Execute the same query using the maxrec parameter instead of the TOP
query = "SELECT objectId, ra, dec, extendedness, mag_r, magerr_r, good " \
        "FROM dp01_dc2_catalogs.object " \
        "WHERE extendedness = 0 " \
        "AND mag_r < 24 " \
        "ORDER by mag_r DESC"
results1 = service.search(query, maxrec=max_rec)
assert len(results1) == max_rec

In [12]:
# Convert the results to pandas data frames and  assert that the 
# contents of the two tables are identical. 
assert_frame_equal(results.to_table().to_pandas(),
                   results1.to_table().to_pandas())

#### 2.2 Cone search around a point with specified radius

We will execute a cone search on the Object table centered on (RA, Dec) = (62.0, -37.0) with a radius of 0.1 degrees and applying a cut on magnitude.
We expect to get 15,670 results.

In [13]:
# Define our reference position on the sky and cone radius in arcseconds
# to use in all following examples
coord = SkyCoord(ra=62.0*u.degree, dec=-37.0*u.degree, frame='icrs')
radius = 0.1 * u.deg

In [14]:
query = "SELECT ra, dec, mag_g, mag_i " \
        "mag_i, mag_g_cModel, mag_r_cModel, mag_i_cModel, " \
        "psFlux_g, psFlux_r, psFlux_i, " \
        "cModelFlux_g, cModelFlux_r, cModelFlux_i, " \
        "tract, patch, extendedness, good, clean " \
        "FROM dp01_dc2_catalogs.object " \
        "WHERE CONTAINS(POINT('ICRS', ra, dec),CIRCLE('ICRS', " \
        + str(coord.ra.value) + ", " + str(coord.dec.value) + ", " \
        + str(radius.value) + " )) = 1"
print(query)

SELECT ra, dec, mag_g, mag_i mag_i, mag_g_cModel, mag_r_cModel, mag_i_cModel, psFlux_g, psFlux_r, psFlux_i, cModelFlux_g, cModelFlux_r, cModelFlux_i, tract, patch, extendedness, good, clean FROM dp01_dc2_catalogs.object WHERE CONTAINS(POINT('ICRS', ra, dec),CIRCLE('ICRS', 62.0, -37.0, 0.1 )) = 1


In [ ]:
# For more detailed analysis of results, converting
# to a pandas dataframe is often very useful
results = service.search(query).to_table().to_pandas()

# Use an assertion to make sure we got the correct number of results.
assert len(results) == 15670

#### 2.3. Cone search joining the results with the truth infomation 

We will now join the results from the above query with the truth information.  We will also add in some quality filters on the match. Objects in the truth-match table that do not have matches in the object table have “match_objectId = -1,” while those with legitimate matches contain the ‘objectId’ of the corresponding object from the object table in “match_objectId.” By requiring this to be greater than or equal to zero, we extract only objects with matches. We also keep only sources satisfying the “is_good_match” flag, which is described in the schema as being “True if this object–truth matching pair satisfies all matching criteria.” (Note that “1” and “TRUE” are equivalent in ADQL.)

With these additional quality filters applied to the matching with the truth information, we only get 14424 results.

In [ ]:
query = "SELECT obj.objectId, obj.ra, obj.dec, obj.mag_g, obj.mag_r, " \
        " obj.mag_i, obj.mag_g_cModel, obj.mag_r_cModel, obj.mag_i_cModel," \
        "obj.psFlux_g, obj.psFlux_r, obj.psFlux_i, obj.cModelFlux_g, " \
        "obj.cModelFlux_r, obj.cModelFlux_i, obj.tract, obj.patch, " \
        "obj.extendedness, obj.good, obj.clean, " \
        "truth.mag_r as truth_mag_r, truth.match_objectId, "\
        "truth.flux_g, truth.flux_r, truth.flux_i, truth.truth_type, " \
        "truth.match_sep, truth.is_variable " \
        "FROM dp01_dc2_catalogs.object as obj " \
        "JOIN dp01_dc2_catalogs.truth_match as truth " \
        "ON truth.match_objectId = obj.objectId " \
        "WHERE CONTAINS(POINT('ICRS', obj.ra, obj.dec),CIRCLE('ICRS', " \
        + str(coord.ra.value) + ", " + str(coord.dec.value) + ", " \
        + str(radius.value) + " )) = 1 " \
        "AND truth.match_objectid >= 0 "\
        "AND truth.is_good_match = 1"

In [ ]:
results = service.search(query).to_table().to_pandas()
assert len(results) == 14424

In [ ]:
# How many of each type in the dataset. 
# The 'truth_type' in the truth_match table is 1= galaxies, 2=stars, 3=SNe. 
n_stars = results[results["truth_type"] == 2].shape[0]
print(f'There are {n_stars} stars out of a total of {len(results)}')
print(f'There are {results[results["truth_type"] == 1].shape[0]} galaxies')
print(f'There are {results[results["truth_type"] == 3].shape[0]} SNe')

### 3. Visualize and analyse the results

Now we will do some interactive analysis with the data we have above. We will use bokeh to create interactive plots so that we can explore the dataset, using multiple panels showing different representations of the same dataset. A selection applied to either panel will highlight the selected points in the other panel.

<a href="https://bokeh.org/">Bokeh Documentation</a> <br>
<a href="https://holoviews.org/">Holoviews Documentation</a>

#### 3.1 Data preparation
The basis for any data visualization is the underlying data. We will prepare ColumnDataSource (CDS) from the data returned by the query above that can be passed directly to bokeh. The CDS is the core of bokeh plots. Bokeh automatically creates a CDS from data passed as python lists or numpy arrays.  CDS are useful as they allow data to be shared between multiple plots and renderers, enabling brushing and linking.  A CDS is essentially a collection of sequences of data that have their own unique column name. 

Getting the data preparation phase right is key to creating powerful visualizations. 

In [ ]:
# Create a python dictionary to store the data from the
# query and pass to the ColumnDataSource
# All columns in a CDS must have the same length
data = dict(ra=results['ra'], dec=results['dec'],
            target_ra=results['ra']-coord.ra.value,
            target_dec=results['dec']-coord.dec.value,
            gmi=results['mag_g_cModel']-results['mag_i_cModel'],
            gmag=results['mag_g_cModel'],
            rmag=results['mag_r_cModel'],
            imag=results['mag_i_cModel']
            )
source = ColumnDataSource(data=data)

# Additional data can be added to the Column Data Source after creation
source.data['objectId'] = results['objectId']

# We will want to filter on the truth type later
# We will convert the truth_type integer to a more descriptive string
object_map = {1: 'galaxy', 2: 'star', 3: 'SNe'}
source.data['truth_type'] = results['truth_type'].map(object_map)

In [ ]:
# Check the truth_type has been updated
source.data['truth_type']

#### 3.2 Color-Magnitude Diagram 
We will use bokeh to plot a color-magnitude (g vs. g-i) diagram making use of the cModel magnitudes. Hover over the points 
in the plot to see their values. 

In [ ]:
# Define the plot asthetics and tools
plot_options = {'plot_height': 400, 'plot_width': 400,
                'tools': ['box_select', 'reset', 'box_zoom', 'help']}
# Define the hover tool
tooltips = [
    ("Col (g-i)", "@gmi"),
    ("Mag (g)", "@gmag"),
    ("Mag (r)", "@rmag"),
    ("Mag (i)", "@imag"),
    ("Type", "@truth_type")
]
hover_tool_cmd = HoverTool(tooltips=tooltips)

In [ ]:
# Create a Colour-Magnitude Diagram, color coding the differnt object types
p = figure(title="Colour - Magnitude Diagram",
           x_axis_label='g-i', y_axis_label='g',
           x_range=(-1.8, 4.3), y_range=(29.5, 16),
           **plot_options)

# Define a palette for the truth types 
truth_type_palette = ['darkred', 'lightgrey', 'blue']
p.add_tools(hover_tool_cmd)
p.circle(x='gmi', y='gmag', source=source,
         size=3, alpha=0.6,
         legend_field="truth_type",
         color=factor_cmap('truth_type', 
                           palette = truth_type_palette,
                           factors = ['star', 'galaxy', 'SNe']),
         hover_color="firebrick")
show(p)

#### 3.3 Color-color (r-i vs. g-r) diagram. 
We will add a color-color (r-i vs. g-r) diagram and make use of the advanced linking features of bokeh to enable brushing and linking between the the color-magnitude diagram and this color-color plot. The CMD in 4.2 is very crowded as it contains 14424 data points. We will now filter on the truth-type to plot stars only.

In [ ]:
# We will now add some additional data to our data structure to
# create a colour-colour diagram
source.data['rmi'] = results['mag_r_cModel'] - results['mag_i_cModel']
source.data['gmr'] = results['mag_g_cModel'] - results['mag_r_cModel']

In [ ]:
# Use a GroupFilter to select rows from the
# CDS that satisfy 'truth_type' stars
stars = CDSView(source=source,
                filters=[GroupFilter(column_name='truth_type', group="star")])

In [ ]:
# Define various options for the plot
plot_options = {'plot_height': 350, 'plot_width': 350,
                'tools': ['box_zoom', 'box_select',
                          'lasso_select', 'reset', 'help']}


# Create the hover tool for these plots
hover_tool = HoverTool(tooltips=[("(RA,DEC)", "(@ra, @dec)"),
                            ("(g-r,g)", "(@gmr, @gmag)"),
                            ("objectId", "@objectId"),
                            ("type", "@truth_type")])


# Spatial plot
title_spatial = f'Spatial centred on (RA,DEC) = ({coord.ra.value},{coord.dec.value})'

fig_spatial = figure(title=title_spatial,
                     x_axis_label="Delta RA", y_axis_label="Delta DEC",
                     **plot_options)
fig_spatial.circle(x='target_ra', y='target_dec', source=source, view=stars,
                   size=4.0, alpha=0.6,
                   color='teal', hover_color='firebrick')
fig_spatial.add_tools(hover_tool)

# Colour magnitude plot
fig_cmag = figure(title="Colour-Magnitude Diagram",
                        x_axis_label="g-r", y_axis_label="g",
                        x_range=(-1.0, 3.5), y_range=(29.5, 16),
                        **plot_options)
fig_cmag.circle(x='gmr', y='gmag', source=source, view=stars,
                      size=4.0, alpha=0.6,
                      color='teal', hover_color='firebrick')
fig_cmag.add_tools(hover_tool)

# Colour colour plot
fig_cc = figure(title="Colour-Colour Diagram",
                      x_axis_label="g-r", y_axis_label="r-i",
                      x_range=(-1.0, 3.5), y_range=(-1.0, 3.5),
                      **plot_options)
fig_cc.circle(x='gmr', y='rmi', source=source, view=stars,
                    size=4.0, alpha=0.6,
                    color='teal', hover_color='firebrick')
fig_cc.add_tools(hover_tool)

# Plot all three on a grid
p = gridplot([[fig_spatial, fig_cmag, fig_cc]])
show(p)

Use the hover tool to see information about individual datapoints (e.g., the object_id). 
This information should appear automatically as you hover the mouse over the datapoints in any of the plots.
Notice the data points highlighted in red on one panel with the hover tool are also highlighted on the other panels.

Click on the selection box icon (with a "+" sign) or the selection lasso icon found in the upper right corner of the figure. 
Use the selection box and selection lasso to make various selections in either panel by clicking and dragging on either panel. 
The selected data points will be displayed in the other panel.

#### 3.4 Comparison with truth information 
Let's compare the measurements from the Object table to the “true” values for some objects. 
We’ll compare the recovered flux to the “true” value that was simulated for each object (as a ratio of the fluxes).

In [ ]:
# Another way that data can be added to the CDS
source.add(results['cModelFlux_r'] / results['flux_r'], name='flux_ratio')
source.add(results['truth_mag_r'], name='truth_mag_r')

In [ ]:
from bokeh.palettes import Dark2
# Create a plot of the distribution of objects on
# sky using the ColumnDataSource's two columns
plot_options = {'plot_height': 300, 'plot_width': 800,
                'tools': ['hover', 'box_select', 'reset', 'help']}

# Define the tooltips and create a HoverTool instance
tooltips = [
    ("Measured/True flux", "@flux_ratio"),
    ("Mag (r)", "@truth_mag_r"),
    ("Type", "@truth_type")
]
hover_tool = HoverTool(tooltips=tooltips)

# Create a new figure 
p = figure(title="Measured/true flux vs true magnitude",
           x_range=(16, 29.5), y_range=(0.15, 2.4),
           x_axis_label='r magnititude (truth)',
           y_axis_label='Measured flux / True flux (r band)',
           **plot_options)
# Add a circle renderer defining several attributes
p.circle(x='truth_mag_r', y='flux_ratio',
         size=3, alpha=0.5, source=source,
         legend_field="truth_type",
         color=factor_cmap('truth_type', 
                           palette=truth_type_palette,
                           factors=['star', 'galaxy', 'SNe']))

# Add the hover tool created above to the figure 
p.add_tools(hover_tool)

# Display
show(p)

### 4.0 Asynchronous TAP queries

So far, we have executed all queries as synchronous queries. This means that the query will continue executing in the notebook until it is finished. You can see when the Jupyter cell is running by the asterisk to the left of the  cell. For synchronous queries, the cell will continue to run until the query completes and the results are returned. The asterisk will then become a number. This is a good option for short queries that take order seconds to minutes.

For longer queries, or for running multiple queries at the same time, an asynchronous query may be more suitable. Asynchronous queries allow you to execute more python while the query runs on the database. Results can be retrieved later on. This is especially important for queries that are long or may return a lot of results. It is also safeguards long queries against network outages or timeouts. 

#### 4.1 Submit and run a job
We will use the cone search joining the results with the truth infomation query from 2.3 and confirm that the results from the asynchronous query are the same as from the synchronous query

In [ ]:
# Create and submit the job. This step does not run the query yet
job = service.submit_job(query)

# # Get the job URL
print('Job URL is', job.url)

# Get the job phase. It will be pending as we have not yet started the job
print('Job phase is', job.phase)

In [ ]:
# Run the job. You will see that the the cell completes executing,
# even though the query is still running
job.run()

In [ ]:
# Use this to tell python to wait for the job to finish if
# you don't want to run anything else while waiting
# The cell will continue executing until the job is finished
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
# A usefull funtion to raise an exception if there was a problem with the query
job.raise_if_error()

In [ ]:
# Once the job completes successfully, you can fetch the results
async_results = job.fetch_result()
assert len(async_results) == 14424
assert_frame_equal(results, async_results.to_table().to_pandas())

#### 4.2 Retrieving the results from a previous asynchronous job
Job results may still be available from previously run queries. You can retrieve these results if you know the URL of the job.

In [ ]:
retrieved_job = retrieve_query(job.url)
previous_results = retrieved_job.fetch_result()
assert len(previous_results) == 14424

#### 4.3 Deleting a job
Once the job is finished and you have retrieved your results, you can delete the job and the results from the server. The results will be deleted automatically after a period of time.

In [ ]:
job.delete()